In [1]:
# Importing mysql.connector and pandas package to work with Sakila Database.

In [2]:
import mysql.connector
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Creating connection with the Sakila Database.

In [3]:
connection = mysql.connector.connect(host = "localhost", user = os.getenv('DB_USER') , passwd = os.getenv('DB_PASSWD'), database = "Sakila")

In [4]:
cursor = connection.cursor()

In [5]:
sakila_tables = pd.read_sql_query("SHOW tables", connection)

In [6]:
# Viewing the tables in the database.

In [7]:
sakila_tables

,Tables_in_sakila
0,actor
1,actor_info
2,address
3,category
4,city
5,country
6,customer
7,customer_list
8,film
9,film_actor


In [8]:
# Extracting table name column as a series.

In [9]:
sakila_tables["Tables_in_sakila"]

0                          actor
1                     actor_info
2                        address
3                       category
4                           city
5                        country
6                       customer
7                  customer_list
8                           film
9                     film_actor
10                 film_category
11                     film_list
12                     film_text
13                     inventory
14                      language
15    nicer_but_slower_film_list
16                       payment
17                        rental
18        sales_by_film_category
19                sales_by_store
20                         staff
21                    staff_list
22                         store
Name: Tables_in_sakila, dtype: object

In [10]:
# Describing the structure (Column names, datatype etc.) of all the tables in the database.

In [11]:
for table in sakila_tables["Tables_in_sakila"]:
    description = pd.read_sql_query("DESCRIBE {}".format(table),connection)
    print(table)
    print(description)

actor
         Field                  Type Null  Key               Default  \
0     actor_id  b'smallint unsigned'   NO  PRI                  None   
1   first_name        b'varchar(45)'   NO                       None   
2    last_name        b'varchar(45)'   NO  MUL                  None   
3  last_update          b'timestamp'   NO       b'CURRENT_TIMESTAMP'   

                                           Extra  
0                                 auto_increment  
1                                                 
2                                                 
3  DEFAULT_GENERATED on update CURRENT_TIMESTAMP  
actor_info
        Field                  Type Null Key Default Extra
0    actor_id  b'smallint unsigned'   NO        b'0'      
1  first_name        b'varchar(45)'   NO        None      
2   last_name        b'varchar(45)'   NO        None      
3   film_info               b'text'  YES        None      
address
         Field                  Type Null  Key               Def

In [12]:
# Quering all the rows in the actor table where first_name starts with "M" and last_name ends with "R"

In [13]:
cursor.execute("""SELECT CONCAT(fil_act.first_name,' ',fil_act.last_name) AS "Full Name"
                 FROM
                     (SELECT first_name, last_name
                      FROM actor
                      WHERE first_name like "M%" AND last_name like "%R"
                      ) AS fil_act
               """)

data1 = pd.DataFrame(cursor.fetchall())
data1.columns = [x[0] for x in cursor.description]
data1

,Full Name
0,MINNIE ZELLWEGER
1,MINNIE KILMER
2,MENA HOPPER
3,MICHAEL BOLGER


In [14]:
# Quering all the rows from rental table where rental_date = 2005-06-14.

In [15]:
cursor.execute("""SELECT c.first_name, c.last_name, time(r.rental_date) AS rental_time
                 FROM customer as c
                  INNER JOIN rental as r
                  ON c.customer_id = r.customer_id
                 WHERE date(r.rental_date) = "2005-06-14" """)
                  

data2 = pd.DataFrame(cursor.fetchall())
data2.columns = [x[0] for x in cursor.description]
data2

,first_name,last_name,rental_time
0,CATHERINE,CAMPBELL,0 days 23:17:03
1,JOYCE,EDWARDS,0 days 23:16:26
2,AMBER,DIXON,0 days 23:42:56
3,JEANETTE,GREENE,0 days 23:54:46
4,MINNIE,ROMERO,0 days 23:00:34
5,GWENDOLYN,MAY,0 days 23:16:27
6,SONIA,GREGORY,0 days 23:50:11
7,MIRIAM,MCKINNEY,0 days 23:07:08
8,CHARLES,KOWALSKI,0 days 23:54:34
9,DANIEL,CABRAL,0 days 23:09:38


In [16]:
# Quering all the rows from rental table where rental_date < 2005-06-14.

In [17]:
cursor.execute("""SELECT c.first_name, c.last_name, time(r.rental_date) as rental_time
                 FROM customer as c
                  INNER JOIN rental as r
                  ON c.customer_id = r.customer_id
                 WHERE date(r.rental_date) < "2005-06-14" """)
                  

data7 = pd.DataFrame(cursor.fetchall())
data7.columns = [x[0] for x in cursor.description]
data7

,first_name,last_name,rental_time
0,MARY,SMITH,0 days 11:30:37
1,MARY,SMITH,0 days 10:35:23
2,PATRICIA,JOHNSON,0 days 00:09:24
3,LINDA,WILLIAMS,0 days 17:17:09
4,LINDA,WILLIAMS,0 days 22:43:55
...,...,...,...
1151,ENRIQUE,FORSYTHE,0 days 11:06:00
1152,ENRIQUE,FORSYTHE,0 days 20:28:42
1153,FREDDIE,DUGGAN,0 days 04:19:28
1154,FREDDIE,DUGGAN,0 days 03:09:28


In [18]:
# Quering all the rows from rental table where rental_date between 2005-06-14 and 2005-06-16.

In [19]:
cursor.execute("""SELECT c.first_name, c.last_name, r.rental_date
                 FROM customer as c
                  INNER JOIN rental as r
                  ON c.customer_id = r.customer_id
                 WHERE r.rental_date BETWEEN "2005-06-14" AND "2005-06-16" """)
                  

data9 = pd.DataFrame(cursor.fetchall())
data9.columns = [x[0] for x in cursor.description]
data9

,first_name,last_name,rental_date
0,JEFFERY,PINSON,2005-06-14 22:53:33
1,ELMER,NOE,2005-06-14 22:55:13
2,MINNIE,ROMERO,2005-06-14 23:00:34
3,MIRIAM,MCKINNEY,2005-06-14 23:07:08
4,DANIEL,CABRAL,2005-06-14 23:09:38
...,...,...,...
359,ELEANOR,HUNT,2005-06-15 23:20:26
360,TANYA,GILBERT,2005-06-15 23:36:37
361,GAIL,KNIGHT,2005-06-15 23:55:27
362,PETER,MENARD,2005-06-15 23:57:20


In [20]:
# Quering all the rows from film table where rating is either G or PG.

In [21]:
cursor.execute("""SELECT title, rating
                 FROM film AS c                  
                 WHERE rating IN ('G','PG') """)
                  

data10 = pd.DataFrame(cursor.fetchall())
data10.columns = [x[0] for x in cursor.description]
data10

,title,rating
0,ACADEMY DINOSAUR,PG
1,ACE GOLDFINGER,G
2,AFFAIR PREJUDICE,G
3,AFRICAN EGG,G
4,AGENT TRUMAN,PG
...,...,...
367,WON DARES,PG
368,WONDERLAND CHRISTMAS,PG
369,WORDS HUNTER,PG
370,WORST BANGER,PG


In [22]:
# Quering all the rows from film table where title contains "PET" (using non-correlated subquery)

In [23]:
cursor.execute("""SELECT title, rating
                 FROM film                  
                 WHERE rating IN (SELECT rating
                                  FROM film
                                  WHERE title like "%PET%") """)
                  

data11 = pd.DataFrame(cursor.fetchall())
data11.columns = [x[0] for x in cursor.description]
data11

,title,rating
0,ACADEMY DINOSAUR,PG
1,ACE GOLDFINGER,G
2,AFFAIR PREJUDICE,G
3,AFRICAN EGG,G
4,AGENT TRUMAN,PG
...,...,...
367,WON DARES,PG
368,WONDERLAND CHRISTMAS,PG
369,WORDS HUNTER,PG
370,WORST BANGER,PG


In [24]:
# Quering all the rows from film table where eighter (rating = G and rental_duration >=7) or (rating = "PG-13" AND rental_duration <4)
# and ordering the result set by rating (ascending) and title (descending).

In [25]:
cursor.execute("""SELECT title, rating, rental_duration
                 FROM film                
                 WHERE (rating = "G" AND rental_duration >=7)
                     OR (rating = "PG-13" AND rental_duration <4)
                 ORDER BY rating ASC, title DESC""")
                  

data5 = pd.DataFrame(cursor.fetchall())
data5.columns = [x[0] for x in cursor.description]
data5

,title,rating,rental_duration
0,WAR NOTTING,G,7
1,WAKE JAWS,G,7
2,TRUMAN CRAZY,G,7
3,SPIKING ELEMENT,G,7
4,REBEL AIRPORT,G,7
...,...,...,...
63,CONFUSED CANDLES,PG-13,3
64,CASPER DRAGONFLY,PG-13,3
65,BILKO ANONYMOUS,PG-13,3
66,BACKLASH UNDEFEATED,PG-13,3


In [26]:
# Ordering by column location in the result set.

In [27]:
cursor.execute("""SELECT title, rating, rental_duration
                 FROM film                
                 WHERE (rating = "G" AND rental_duration >=7)
                     OR (rating = "PG-13" AND rental_duration <4)
                 ORDER BY 2 ASC, 1 DESC""")
                  

data6 = pd.DataFrame(cursor.fetchall())
data6.columns = [x[0] for x in cursor.description]
data6

,title,rating,rental_duration
0,WAR NOTTING,G,7
1,WAKE JAWS,G,7
2,TRUMAN CRAZY,G,7
3,SPIKING ELEMENT,G,7
4,REBEL AIRPORT,G,7
...,...,...,...
63,CONFUSED CANDLES,PG-13,3
64,CASPER DRAGONFLY,PG-13,3
65,BILKO ANONYMOUS,PG-13,3
66,BACKLASH UNDEFEATED,PG-13,3


In [28]:
# Quering number of rentals per customer.

In [29]:
cursor.execute("""SELECT c.first_name, c.last_name, count(*) AS "Number of Rentals"
                 FROM customer AS c
                  INNER JOIN rental AS r
                  ON c.customer_id = r.customer_id
                 GROUP BY c.first_name,c.last_name """)
                  

data3 = pd.DataFrame(cursor.fetchall())
data3.columns = [x[0] for x in cursor.description]
data3

,first_name,last_name,Number of Rentals
0,MARY,SMITH,32
1,PATRICIA,JOHNSON,27
2,LINDA,WILLIAMS,26
3,BARBARA,JONES,22
4,ELIZABETH,BROWN,38
...,...,...,...
594,TERRENCE,GUNDERSON,30
595,ENRIQUE,FORSYTHE,28
596,FREDDIE,DUGGAN,25
597,WADE,DELVALLE,22


In [30]:
#Quering customers who rented atleast 40 times.

In [31]:
cursor.execute("""SELECT c.first_name, c.last_name, count(*) AS number_of_rentals
                 FROM customer AS c
                  INNER JOIN rental AS r
                  ON c.customer_id = r.customer_id
                 GROUP BY c.first_name,c.last_name
                 HAVING count(*) >= 40""")
                  

data4 = pd.DataFrame(cursor.fetchall())
data4.columns = [x[0] for x in cursor.description]
data4

,first_name,last_name,number_of_rentals
0,TAMMY,SANDERS,41
1,CLARA,SHAW,42
2,ELEANOR,HUNT,46
3,SUE,PETERS,40
4,MARCIA,DEAN,42
5,WESLEY,BULL,40
6,KARL,SEAL,45


In [32]:
# Quering customers who spent atleast $100 on renting.

In [33]:
cursor.execute("""SELECT c.first_name, c.last_name, sum(p.amount) AS total_payment
                 FROM customer AS c
                  INNER JOIN payment AS p
                  ON c.customer_id = p.customer_id
                 GROUP BY c.first_name,c.last_name
                 HAVING SUM(p.amount) >= 100""")
                  

data8 = pd.DataFrame(cursor.fetchall())
data8.columns = [x[0] for x in cursor.description]
data8

,first_name,last_name,total_payment
0,MARY,SMITH,118.68
1,PATRICIA,JOHNSON,128.73
2,LINDA,WILLIAMS,135.74
3,ELIZABETH,BROWN,144.62
4,MARIA,MILLER,151.67
...,...,...,...
390,KENT,ARSENAULT,134.73
391,TERRANCE,ROUSH,111.71
392,RENE,MCALISTER,113.74
393,EDUARDO,HIATT,130.73


In [34]:
# Quering total payment received in May, June and July months as a single row using CASE clause.

In [35]:
cursor.execute("""SELECT
                    SUM(CASE WHEN MONTH(rental_date) = 5 THEN 1 ELSE 0 END) AS may_rental,
                    SUM(CASE WHEN MONTH(rental_date) = 6 THEN 1 ELSE 0 END) AS june_rental,
                    SUM(CASE WHEN MONTH(rental_date) = 7 THEN 1 ELSE 0 END) AS july_rental                 
                 FROM rental
                """)
                  

data13 = pd.DataFrame(cursor.fetchall())
data13.columns = [x[0] for x in cursor.description]
data13

,may_rental,june_rental,july_rental
0,1156,2311,6709


In [36]:
# Determining if a particular actor acted in atleast one film with G, PG or NC-17 ratings or not.

In [37]:
cursor.execute("""SELECT a.first_name, a.last_name,
                    CASE WHEN EXISTS
                        (SELECT 1
                         FROM film_actor AS fa
                             INNER JOIN film AS f
                             ON fa.film_id = f.film_id
                             WHERE a.actor_id = fa.actor_id
                                 AND rating = "G") THEN "Y" ELSE "N" END AS g_actor,
                    CASE WHEN EXISTS
                        (SELECT 1
                         FROM film_actor AS fa
                             INNER JOIN film AS f
                             ON fa.film_id = f.film_id
                             WHERE a.actor_id = fa.actor_id
                                 AND rating = "PG") THEN "Y" ELSE "N" END AS pg_actor,
                    CASE WHEN EXISTS
                        (SELECT 1
                         FROM film_actor AS fa
                             INNER JOIN film AS f
                             ON fa.film_id = f.film_id
                             WHERE a.actor_id = fa.actor_id
                                 AND rating = "NC-17") THEN "Y" ELSE "N" END AS nc17_actor               
                  FROM actor AS a
                  WHERE a.first_name like "S%" OR a.last_name like "S%"
                """)
                  

data14 = pd.DataFrame(cursor.fetchall())
data14.columns = [x[0] for x in cursor.description]
data14

,first_name,last_name,g_actor,pg_actor,nc17_actor
0,JOE,SWANK,Y,Y,Y
1,SANDRA,KILMER,Y,Y,Y
2,CAMERON,STREEP,Y,Y,Y
3,SANDRA,PECK,Y,Y,Y
4,SISSY,SOBIESKI,Y,Y,N
5,NICK,STALLONE,Y,Y,Y
6,SEAN,WILLIAMS,Y,Y,Y
7,GROUCHO,SINATRA,Y,Y,Y
8,SCARLETT,DAMON,Y,Y,Y
9,SPENCER,PECK,Y,Y,Y


In [38]:
# Calculating total payment, number of rentals and average payment done by each customer. The result set is limited to 50 rows.

In [39]:
cursor.execute("""SELECT c.first_name, c.last_name, SUM(p.amount) AS total_payment,
                        COUNT(p.amount) AS num_payments,
                        SUM(p.amount)/
                            CASE WHEN COUNT(p.amount) = 0 THEN 1 ELSE
                                COUNT(p.amount) END AS avg_payment                  
                  FROM customer AS c
                      INNER JOIN payment AS p
                      ON c.customer_id = p.customer_id
                  GROUP BY c.first_name, c.last_name
                  LIMIT 50
                """)
                  

data15 = pd.DataFrame(cursor.fetchall())
data15.columns = [x[0] for x in cursor.description]
data15

,first_name,last_name,total_payment,num_payments,avg_payment
0,MARY,SMITH,118.68,32,3.708750
1,PATRICIA,JOHNSON,128.73,27,4.767778
2,LINDA,WILLIAMS,135.74,26,5.220769
3,BARBARA,JONES,81.78,22,3.717273
4,ELIZABETH,BROWN,144.62,38,3.805789
5,JENNIFER,DAVIS,93.72,28,3.347143
6,MARIA,MILLER,151.67,33,4.596061
7,SUSAN,WILSON,92.76,24,3.865000
8,MARGARET,MOORE,89.77,23,3.903043
9,DOROTHY,TAYLOR,99.75,25,3.990000


In [40]:
# Calculating monthly total payment, max payment acrocss the months and max quarterly payment.

In [41]:
cursor.execute("""SELECT quarter(p.payment_date) AS quarter, monthname(p.payment_date) AS month,
                    SUM(p.amount) AS total_payment,
                    MAX(SUM(p.amount)) OVER() AS max_overall_payment,
                    MAX(SUM(p.amount)) OVER(PARTITION BY quarter(p.payment_date)) AS quarterly_max
                  FROM payment AS p
                  WHERE YEAR(p.payment_date) = 2005
                  GROUP BY quarter(p.payment_date), monthname(p.payment_date)
                """)
                  

data16 = pd.DataFrame(cursor.fetchall())
data16.columns = [x[0] for x in cursor.description]
data16

,quarter,month,total_payment,max_overall_payment,quarterly_max
0,2,May,4824.43,28373.89,9631.88
1,2,June,9631.88,28373.89,9631.88
2,3,July,28373.89,28373.89,28373.89
3,3,August,24072.13,28373.89,28373.89


In [42]:
cursor.execute("""SELECT QUARTER(p.payment_date) AS quarter, MONTHNAME(p.payment_date) AS month,
                    SUM(p.amount) AS total_payment,
                    RANK() OVER(PARTITION BY QUARTER(p.payment_date)
                        ORDER BY SUM(p.amount) DESC) AS quarterly_rank                
                  FROM payment AS p
                  WHERE YEAR(p.payment_date) = 2005
                  GROUP BY QUARTER(p.payment_date), MONTHNAME(p.payment_date)
                  ORDER BY 1, MONTH(p.payment_date)
                """)
                  

data17 = pd.DataFrame(cursor.fetchall())
data17.columns = [x[0] for x in cursor.description]
data17

,quarter,month,total_payment,quarterly_rank
0,2,May,4824.43,2
1,2,June,9631.88,1
2,3,July,28373.89,1
3,3,August,24072.13,2


In [43]:
# Ranking of customers based on number of rentals during the month.

In [44]:
cursor.execute("""SELECT r.customer_id, monthname(r.rental_date),
                    COUNT(*) AS num_rentals,
                    RANK() OVER(PARTITION BY MONTHNAME(r.rental_date)
                        ORDER BY COUNT(*) DESC) AS rank_rnk                
                  FROM rental AS r
                  GROUP BY r.customer_id, MONTHNAME(r.rental_date)
                  ORDER BY MONTH(r.rental_date),3 DESC
               """)
                  

data18 = pd.DataFrame(cursor.fetchall())
data18.columns = [x[0] for x in cursor.description]
data18

,customer_id,monthname(r.rental_date),num_rentals,rank_rnk
0,75,February,3,1
1,457,February,2,2
2,560,February,2,2
3,155,February,2,2
4,269,February,2,2
...,...,...,...,...
2461,429,August,3,589
2462,281,August,2,596
2463,164,August,2,596
2464,191,August,2,596


In [45]:
# Top 5 customers every month based on number of rentals.

In [46]:
cursor.execute("""SELECT customer_id, month_name, num_rentals, rank_rnk

                  FROM
                  
                  (SELECT r.customer_id, MONTHNAME(r.rental_date) AS month_name,
                    COUNT(*) AS num_rentals,
                    RANK() OVER(PARTITION BY MONTHNAME(r.rental_date)
                        ORDER BY COUNT(*) DESC) AS rank_rnk                
                  FROM rental AS r
                  GROUP BY r.customer_id, MONTHNAME(r.rental_date)
                  ORDER BY MONTH(r.rental_date),3 DESC) AS cust_ranking
                  
                  WHERE rank_rnk <= 5
                  
                  ORDER BY month_name, num_rentals DESC, rank_rnk
               """)
                  

data19 = pd.DataFrame(cursor.fetchall())
data19.columns = [x[0] for x in cursor.description]
data19

,customer_id,month_name,num_rentals,rank_rnk
0,119,August,18,1
1,569,August,18,1
2,15,August,18,1
3,148,August,18,1
4,266,August,17,5
...,...,...,...,...
63,251,May,6,4
64,239,May,6,4
65,161,May,6,4
66,274,May,6,4


In [47]:
#Calculating monthly payment, grand total of all months and monthly_total to grand_total percentage.

In [48]:
cursor.execute("""SELECT MONTHNAME(payment_date) AS month_name,
                    SUM(amount) AS monthly_total,
                    SUM(SUM(amount)) OVER() AS grand_total,
                    ROUND(SUM(amount)/SUM(SUM(amount)) OVER() * 100,2) AS pct
                    
                  
                  FROM payment
                  
                  GROUP BY MONTHNAME(payment_date)
                  ORDER BY MONTH(payment_date)         
               """)
                  

data20 = pd.DataFrame(cursor.fetchall())
data20.columns = [x[0] for x in cursor.description]
data20

,month_name,monthly_total,grand_total,pct
0,February,514.18,67416.51,0.76
1,May,4824.43,67416.51,7.16
2,June,9631.88,67416.51,14.29
3,July,28373.89,67416.51,42.09
4,August,24072.13,67416.51,35.71


In [49]:
#Describing which month had the highest, lowest and in middle payment amounts.

In [50]:
cursor.execute("""SELECT MONTHNAME(payment_date) AS month_name,
                    SUM(amount) AS monthly_total,
                    CASE SUM(amount)
                        WHEN MAX(SUM(amount)) OVER() THEN "Highest"
                        WHEN MIN(SUM(amount)) OVER() THEN "Lowest"
                        ELSE "Middle"
                    END AS descriptor
                    
                  
                  FROM payment
                  
                  GROUP BY MONTHNAME(payment_date)
                  ORDER BY MONTH(payment_date)         
               """)
                  

data21 = pd.DataFrame(cursor.fetchall())
data21.columns = [x[0] for x in cursor.description]
data21

,month_name,monthly_total,descriptor
0,February,514.18,Lowest
1,May,4824.43,Middle
2,June,9631.88,Middle
3,July,28373.89,Highest
4,August,24072.13,Middle


In [51]:
# Calculating rolling total payments by week

In [52]:
cursor.execute("""SELECT YEARWEEK(payment_date) as payment_week,
                    SUM(amount) AS weekly_payment,
                    SUM(SUM(amount))
                        OVER (ORDER BY YEARWEEK(payment_date)
                            rows unbounded preceding) AS rollings_total
                    
                  
                  FROM payment
                  
                  GROUP BY YEARWEEK(payment_date)
                  ORDER BY YEARWEEK(payment_date)         
               """)
                  

data22 = pd.DataFrame(cursor.fetchall())
data22.columns = [x[0] for x in cursor.description]
data22

,payment_week,weekly_payment,rollings_total
0,200521,2847.18,2847.18
1,200522,1977.25,4824.43
2,200524,5605.42,10429.85
3,200525,4026.46,14456.31
4,200527,8490.83,22947.14
5,200528,5983.63,28930.77
6,200530,11031.22,39961.99
7,200531,8412.07,48374.06
8,200533,10619.11,58993.17
9,200534,7909.16,66902.33


In [53]:
# Calculating 3 week roling total (1 week prior, current week and 1 week following)

In [54]:
cursor.execute("""SELECT YEARWEEK(payment_date) as payment_week,
                    SUM(amount) AS weekly_payment,
                    ROUND(AVG(SUM(amount))
                        OVER (ORDER BY YEARWEEK(payment_date)
                            ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING),2) AS 3wk_rolling_total
                    
                  
                  FROM payment
                  
                  GROUP BY YEARWEEK(payment_date)
                  ORDER BY YEARWEEK(payment_date)         
               """)
                  

data23 = pd.DataFrame(cursor.fetchall())
data23.columns = [x[0] for x in cursor.description]
data23

,payment_week,weekly_payment,3wk_rolling_total
0,200521,2847.18,2412.22
1,200522,1977.25,3476.62
2,200524,5605.42,3869.71
3,200525,4026.46,6040.90
4,200527,8490.83,6166.97
5,200528,5983.63,8501.89
6,200530,11031.22,8475.64
7,200531,8412.07,10020.80
8,200533,10619.11,8980.11
9,200534,7909.16,6347.48


In [55]:
# Calculating previous and next week's total using LEAD and LAG reporting functions.

In [56]:
cursor.execute("""SELECT YEARWEEK(payment_date) as payment_week,
                    SUM(amount) AS weekly_payment,
                    LAG(SUM(amount),1) 
                        OVER(ORDER BY YEARWEEK(payment_date)) AS prev_week_payment,
                    LEAD(SUM(amount),1) 
                        OVER(ORDER BY YEARWEEK(payment_date)) AS next_week_payment                    
                    
                  FROM payment
                  
                  GROUP BY YEARWEEK(payment_date)
                  ORDER BY YEARWEEK(payment_date)         
               """)
                  

data24 = pd.DataFrame(cursor.fetchall())
data24.columns = [x[0] for x in cursor.description]
data24

,payment_week,weekly_payment,prev_week_payment,next_week_payment
0,200521,2847.18,None,1977.25
1,200522,1977.25,2847.18,5605.42
2,200524,5605.42,1977.25,4026.46
3,200525,4026.46,5605.42,8490.83
4,200527,8490.83,4026.46,5983.63
5,200528,5983.63,8490.83,11031.22
6,200530,11031.22,5983.63,8412.07
7,200531,8412.07,11031.22,10619.11
8,200533,10619.11,8412.07,7909.16
9,200534,7909.16,10619.11,514.18
